<a href="https://colab.research.google.com/github/gcochlar/Bootcamp_DataScience/blob/main/Modulo_06/Bootcamp_Mod6_Aula01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src='https://drive.google.com/uc?id=16O9eMrtSeRDnDpZgmneXj34eIClCntvg'>

# ***Data Science*** **Aplicado em Finanças**

---

Esse notebook servirá para o acompanhamento das aulas do Módulo 06 do ***Bootcamp de Data Science Aplicada*** realizado pela Alura.

O Módulo 6 é um módulo extra especialmente desenvolvido para este *Bootcamp* e é voltado para a área financeira, mais especificamente para a área de análise de crédito.

Além do desenvolvimento do modelo de *machine learning* em si, também está no escopo o *deploy* através do *streamlit*.

---

>Agora chegou o momento de consolidar seu conhecimento em uma nova área. Vamos sair do assunto saúde e desenvolver um projeto completo de Data Science aplicado a finanças. Será mais um projeto incrível para complementar seu portfólio e principalmente para te ensinar assuntos específicos desta área tão relevante.
>
>Então prepare os equipamentos porque nosso mergulho será em outros mares!
---

## **Aula 01 - Contextualização do Mercado Financeiro**
---


A aula começou falando dos pilares que temos na cadeia de crédito. São eles:

* Prospecção
* Concessão
* Gestão de Risco
* Recuperação

O modelo que iremos trabalhar estará no pilar **CONCESSÃO**. Vamos trabalhar o modelo de decisão que irá avaliar a concessão ou não do crédito solicitado.

Na sequência fomos informados que a base que iremos usar é uma adaptação de uma base que foi disponibilizada em um desafio do **Kaggle**.

São duas bases de dados, com os clientes cadastrados e com os empréstimos aprovados.

Baixei as bases e fiz *upload* para o meu repositório do **GitHub**.

In [1]:
import pandas as pd
import numpy as np


In [2]:
url_cadastrados = 'https://github.com/gcochlar/Bootcamp_DataScience/blob/main/Modulo_06/dados/clientes_cadastrados.zip?raw=true'
url_aprovados = 'https://github.com/gcochlar/Bootcamp_DataScience/blob/main/Modulo_06/dados/clientes_aprovados.zip?raw=true'

Vamos começar importando a base de clientes cadastrados.

In [3]:
dados_cadastrados = pd.read_csv(url_cadastrados, compression='zip')
dados_cadastrados.head()

,ID_Cliente,Genero,Tem_Carro,Tem_Casa_Propria,Qtd_Filhos,Rendimento_Anual,Categoria_de_renda,Grau_Escolaridade,Estado_Civil,Moradia,Idade,Tem_Celular,Tem_telefone_trabalho,Tem_telefone_fixo,Tem_email,Ocupacao,Tamanho_Familia,Anos_empregado
0,5008804,H,1,1,0,427500.0,Empregado,Ensino superior,Uniao estavel,Alugado,33.0,1,1,0,0,Outros,2.0,12.4
1,5008805,H,1,1,0,427500.0,Empregado,Ensino superior,Uniao estavel,Alugado,33.0,1,1,0,0,Outros,2.0,12.4
2,5008806,H,1,1,0,112500.0,Empregado,Ensino medio,Casado,Casa propria,59.0,1,0,0,0,Seguranca,2.0,3.1
3,5008808,M,0,1,0,270000.0,Associado comercial,Ensino medio,Solteiro,Casa propria,52.0,1,0,1,1,Venda,1.0,8.4
4,5008809,M,0,1,0,270000.0,Associado comercial,Ensino medio,Solteiro,Casa propria,52.0,1,0,1,1,Venda,1.0,8.4


In [5]:
dados_cadastrados.shape

(438557, 18)

Essa primeira base, de clientes cadastrados, é bem extensa.

Tem quase 440 mil registros com 18 variáveis diferentes.

Imediatamente chama a atenção a coluna **`ID_Cliente`** que deve servir como chave de identificação de cada cliente, mas isso precisa ser confirmado.

Também chama a atenção o fato de termos linhas aparentemente duplicadas (0 e 1 ou 3 e 4, por exemplo).

Vamos importar e olhar também nossa base de empréstimos aprovados.

In [4]:
dados_aprovados = pd.read_csv(url_aprovados, compression='zip')
dados_aprovados.head()

,ID_Cliente,Mes_referencia,Faixa_atraso
0,5001711,0,nenhum empréstimo
1,5001711,-1,1-29 dias
2,5001711,-2,1-29 dias
3,5001711,-3,1-29 dias
4,5001712,0,pagamento realizado


In [33]:
dados_aprovados.shape

(1048575, 3)

A base de empréstimos aprovados também é bem extensa, com mais de 1 milhão de registros, apesar de contar com apenas 3 variáveis.

Também conta com uma coluna **`ID_Cliente`** mas que já podemos ver que apresenta duplicidades. Deverá servir para conectar as duas bases de dados.

Vamos focar, inicialmente, na análise exploratória da base de clientes cadastrados.

In [8]:
dados_cadastrados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 438557 entries, 0 to 438556
Data columns (total 18 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   ID_Cliente             438557 non-null  int64  
 1   Genero                 438557 non-null  object 
 2   Tem_Carro              438557 non-null  int64  
 3   Tem_Casa_Propria       438557 non-null  int64  
 4   Qtd_Filhos             438557 non-null  int64  
 5   Rendimento_Anual       438557 non-null  float64
 6   Categoria_de_renda     438557 non-null  object 
 7   Grau_Escolaridade      438557 non-null  object 
 8   Estado_Civil           438557 non-null  object 
 9   Moradia                438557 non-null  object 
 10  Idade                  438557 non-null  float64
 11  Tem_Celular            438557 non-null  int64  
 12  Tem_telefone_trabalho  438557 non-null  int64  
 13  Tem_telefone_fixo      438557 non-null  int64  
 14  Tem_email              438557 non-nu

Vamos tentar confirmar nossa ideia de que a coluna **`ID_Cliente`** pode ser usada como identificador único de nossos clientes.

In [9]:
dados_cadastrados.ID_Cliente.value_counts()

7603224    2
7742853    2
7836711    2
7091721    2
7022327    2
          ..
5372264    1
6412652    1
6410605    1
6367602    1
6291456    1
Name: ID_Cliente, Length: 438510, dtype: int64

O fato de termos ocorrências duplas de alguns **IDs** indica algum problema em nossa base de dados.

Vamos verificar se temos registros duplicados, ou seja, linhas completamente iguais.

In [10]:
dados_cadastrados.duplicated().sum()

0

Não temos linhas duplicadas, mas a duplicidade de **IDs** irá nos atrapalhar mais adiante.

Precisamos verificar qual o tamanho do nosso problema. Em uma base de quase 440 mil registros, quantos estão duplicados?

In [16]:
dados_cadastrados[dados_cadastrados.ID_Cliente.duplicated(keep=False)].sort_values(by='ID_Cliente')

,ID_Cliente,Genero,Tem_Carro,Tem_Casa_Propria,Qtd_Filhos,Rendimento_Anual,Categoria_de_renda,Grau_Escolaridade,Estado_Civil,Moradia,Idade,Tem_Celular,Tem_telefone_trabalho,Tem_telefone_fixo,Tem_email,Ocupacao,Tamanho_Familia,Anos_empregado
426818,7022197,H,1,1,3,135000.0,Empregado,Ensino medio,Casado,Casa propria,33.0,1,0,0,1,Construcao Civil,5.0,2.0
425023,7022197,M,0,1,0,450000.0,Associado comercial,Ensino superior,Separado,Casa propria,54.0,1,0,0,1,Outros,1.0,4.9
431545,7022327,M,0,1,0,135000.0,Associado comercial,Ensino medio,Solteiro,Casa propria,40.0,1,0,0,0,Alta tecnologia,1.0,14.5
431911,7022327,H,1,1,0,256500.0,Associado comercial,Ensino superior,Casado,Casa propria,59.0,1,0,0,1,Equipe principal,2.0,4.6
425486,7023108,H,1,1,1,67500.0,Empregado,Ensino medio,Casado,Casa propria,42.0,1,1,0,0,Equipe principal,3.0,4.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426563,7836711,M,0,1,2,292500.0,Empregado,Ensino superior,Casado,Casa propria,38.0,1,0,1,0,Contabilidade,4.0,12.0
421464,7836971,H,1,0,1,157500.0,Empregado,Ensino medio,Casado,Casa propria,38.0,1,0,0,0,Outros,3.0,15.1
428620,7836971,M,0,1,0,103500.0,Empregado,Ensino medio,Uniao estavel,Casa propria,37.0,1,0,1,0,Venda,2.0,7.7
422068,7838075,H,0,1,0,337500.0,Associado comercial,Ensino medio,Casado,Casa propria,50.0,1,0,0,1,Motorista,2.0,3.5


Temos 94 registros duplicados. Isso representa em torno de 0,02% de nossa base.

Inicialmente, podemos desprezar esses registros para avançarmos em nossa análise.

In [27]:
id_repetidos = dados_cadastrados[dados_cadastrados.ID_Cliente.duplicated(keep=False)].sort_values(by='ID_Cliente')['ID_Cliente']
id_repetidos

426818    7022197
425023    7022197
431545    7022327
431911    7022327
425486    7023108
           ...   
426563    7836711
421464    7836971
428620    7836971
422068    7838075
423702    7838075
Name: ID_Cliente, Length: 94, dtype: int64

In [28]:
dados_cad_id_unicos = dados_cadastrados.drop(id_repetidos.index)
dados_cad_id_unicos

,ID_Cliente,Genero,Tem_Carro,Tem_Casa_Propria,Qtd_Filhos,Rendimento_Anual,Categoria_de_renda,Grau_Escolaridade,Estado_Civil,Moradia,Idade,Tem_Celular,Tem_telefone_trabalho,Tem_telefone_fixo,Tem_email,Ocupacao,Tamanho_Familia,Anos_empregado
0,5008804,H,1,1,0,427500.0,Empregado,Ensino superior,Uniao estavel,Alugado,33.0,1,1,0,0,Outros,2.0,12.4
1,5008805,H,1,1,0,427500.0,Empregado,Ensino superior,Uniao estavel,Alugado,33.0,1,1,0,0,Outros,2.0,12.4
2,5008806,H,1,1,0,112500.0,Empregado,Ensino medio,Casado,Casa propria,59.0,1,0,0,0,Seguranca,2.0,3.1
3,5008808,M,0,1,0,270000.0,Associado comercial,Ensino medio,Solteiro,Casa propria,52.0,1,0,1,1,Venda,1.0,8.4
4,5008809,M,0,1,0,270000.0,Associado comercial,Ensino medio,Solteiro,Casa propria,52.0,1,0,1,1,Venda,1.0,8.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438552,6840104,H,0,1,0,135000.0,Pensionista,Ensino medio,Separado,Casa propria,62.0,1,0,0,0,Outros,1.0,-1000.7
438553,6840222,M,0,0,0,103500.0,Empregado,Ensino medio,Solteiro,Casa propria,44.0,1,0,0,0,Construcao Civil,1.0,8.2
438554,6841878,M,0,0,0,54000.0,Associado comercial,Ensino superior,Solteiro,Mora com os pais,22.0,1,1,0,0,Venda,1.0,1.0
438555,6842765,M,0,1,0,72000.0,Pensionista,Ensino medio,Casado,Casa propria,59.0,1,0,0,0,Outros,2.0,-1000.7


In [82]:
(dados_cad_id_unicos.ID_Cliente.value_counts() > 1).sum()

0

Agora estamos com um novo *dataframe*, sem nenhum **ID** duplicado.

Conforme mencionado antes, nossa base aparentava ter dados em duplicidade também nas demais colunas, ou seja, clientes com vários **IDs** diferentes.

Vamos verificar qual o impacto disso em nosso *dataset*.

In [25]:
dados_cadastrados.drop(['ID_Cliente'], axis=1).duplicated().sum()

353636

Aqui a situação é mais complexa.

Temos mais de 350 mil registros idênticos em nossa base, mais de 80%.

Não podemos desprezar esses dados sem investigar melhor o que pode ter acontecido em nossa base.

Vamos prosseguir com a base que fizemos apenas retirando os **IDs** duplicados e verificar a existência de dados nulos em alguma coluna.

In [30]:
# Verificação de dados nulos na base

dados_cad_id_unicos.isnull().sum()

ID_Cliente               0
Genero                   0
Tem_Carro                0
Tem_Casa_Propria         0
Qtd_Filhos               0
Rendimento_Anual         0
Categoria_de_renda       0
Grau_Escolaridade        0
Estado_Civil             0
Moradia                  0
Idade                    0
Tem_Celular              0
Tem_telefone_trabalho    0
Tem_telefone_fixo        0
Tem_email                0
Ocupacao                 0
Tamanho_Familia          0
Anos_empregado           0
dtype: int64

Não temos dados nulos, podemos prosseguir.

In [31]:
dados_cad_id_unicos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 438463 entries, 0 to 438556
Data columns (total 18 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   ID_Cliente             438463 non-null  int64  
 1   Genero                 438463 non-null  object 
 2   Tem_Carro              438463 non-null  int64  
 3   Tem_Casa_Propria       438463 non-null  int64  
 4   Qtd_Filhos             438463 non-null  int64  
 5   Rendimento_Anual       438463 non-null  float64
 6   Categoria_de_renda     438463 non-null  object 
 7   Grau_Escolaridade      438463 non-null  object 
 8   Estado_Civil           438463 non-null  object 
 9   Moradia                438463 non-null  object 
 10  Idade                  438463 non-null  float64
 11  Tem_Celular            438463 non-null  int64  
 12  Tem_telefone_trabalho  438463 non-null  int64  
 13  Tem_telefone_fixo      438463 non-null  int64  
 14  Tem_email              438463 non-nu

### **Desafio 1 - Explorar IDs Repetidos (Atualização de Cadastro? Erro?)**
---



Primeiro vamos gerar um *dataframe* só com os 94 clientes que foram desconsiderados da nossa base acima.

In [44]:
dados_cad_id_repetidos = dados_cadastrados[dados_cadastrados.ID_Cliente.isin(id_repetidos)].sort_values(by='ID_Cliente')
dados_cad_id_repetidos

,ID_Cliente,Genero,Tem_Carro,Tem_Casa_Propria,Qtd_Filhos,Rendimento_Anual,Categoria_de_renda,Grau_Escolaridade,Estado_Civil,Moradia,Idade,Tem_Celular,Tem_telefone_trabalho,Tem_telefone_fixo,Tem_email,Ocupacao,Tamanho_Familia,Anos_empregado
426818,7022197,H,1,1,3,135000.0,Empregado,Ensino medio,Casado,Casa propria,33.0,1,0,0,1,Construcao Civil,5.0,2.0
425023,7022197,M,0,1,0,450000.0,Associado comercial,Ensino superior,Separado,Casa propria,54.0,1,0,0,1,Outros,1.0,4.9
431545,7022327,M,0,1,0,135000.0,Associado comercial,Ensino medio,Solteiro,Casa propria,40.0,1,0,0,0,Alta tecnologia,1.0,14.5
431911,7022327,H,1,1,0,256500.0,Associado comercial,Ensino superior,Casado,Casa propria,59.0,1,0,0,1,Equipe principal,2.0,4.6
425486,7023108,H,1,1,1,67500.0,Empregado,Ensino medio,Casado,Casa propria,42.0,1,1,0,0,Equipe principal,3.0,4.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426563,7836711,M,0,1,2,292500.0,Empregado,Ensino superior,Casado,Casa propria,38.0,1,0,1,0,Contabilidade,4.0,12.0
421464,7836971,H,1,0,1,157500.0,Empregado,Ensino medio,Casado,Casa propria,38.0,1,0,0,0,Outros,3.0,15.1
428620,7836971,M,0,1,0,103500.0,Empregado,Ensino medio,Uniao estavel,Casa propria,37.0,1,0,1,0,Venda,2.0,7.7
422068,7838075,H,0,1,0,337500.0,Associado comercial,Ensino medio,Casado,Casa propria,50.0,1,0,0,1,Motorista,2.0,3.5


In [46]:
dados_cad_id_repetidos.drop('ID_Cliente', axis=1).duplicated().sum()

0

Desprezando o **```ID_Cliente```** desses 94 registros com **ID** duplicado, podemos ver que não temos nenhuma duplicidade do restante dos dados.

Isso indica que temos algum tipo de erro na atribuição desses **IDs**, permitindo que não seja uma chave única em nossa base.

É necessário verificar se foi um erro na extração ou se o erro também está presente no sistema de foram extraídos os dados.

### **Desafio 2 - Explorar as Colunas (*features*) e Identificar se são Contínuas, Categóricas, Binárias**
---



In [47]:
cols_int64 = dados_cad_id_unicos.select_dtypes('int64').columns
cols_float64 = dados_cad_id_unicos.select_dtypes('float64').columns
cols_object = var_float64 = dados_cad_id_unicos.select_dtypes('object').columns

In [55]:
for var in cols_int64:
    print(var,'- Valores Únicos:', str(np.sort(dados_cad_id_unicos[var].unique())))

ID_Cliente - Valores Únicos: [5008804 5008805 5008806 ... 7999738 7999784 7999952]
Tem_Carro - Valores Únicos: [0 1]
Tem_Casa_Propria - Valores Únicos: [0 1]
Qtd_Filhos - Valores Únicos: [ 0  1  2  3  4  5  6  7  9 12 14 19]
Tem_Celular - Valores Únicos: [1]
Tem_telefone_trabalho - Valores Únicos: [0 1]
Tem_telefone_fixo - Valores Únicos: [0 1]
Tem_email - Valores Únicos: [0 1]


In [78]:
dados_cad_id_unicos[dados_cad_id_unicos.Qtd_Filhos.isin([12,14,19])].shape

(8, 18)

As colunas que estão com o tipo **`int64`** são, em sua maioria, binárias.

As exceções são:
* **ID_Cliente:** que já vimos que é o identificador de cliente
* **Qtd_Filhos:** que varia de 0 à 19 (e esses números excessivamente altos podem indicar erros de preenchimento)
* **Tem_Celular:** que é igual a 1 para todos, podendo ser desconsiderada

In [58]:
for var in cols_float64:
    print(var,'- Range:', str(dados_cad_id_unicos[var].min()), 'à', str(dados_cad_id_unicos[var].max()))
    print('          - Média e Desvio Padrão:', str(dados_cad_id_unicos[var].mean()), 'e', str(dados_cad_id_unicos[var].std()))

Rendimento_Anual - Range: 26100.0 à 6750000.0
          - Média e Desvio Padrão: 187522.009381567 e 110083.15376904178
Idade - Range: 21.0 à 69.0
          - Média e Desvio Padrão: 43.83403160585956 e 11.472814834775818
Tamanho_Familia - Range: 1.0 à 20.0
          - Média e Desvio Padrão: 2.1944611061822776 e 0.8971944827602194
Anos_empregado - Range: -1000.7 à 48.0
          - Média e Desvio Padrão: -165.9360358800694 e 380.20113080357845


In [66]:
dados_cad_id_unicos[dados_cad_id_unicos.Anos_empregado == dados_cad_id_unicos.Anos_empregado.min()].shape

(75314, 18)

In [79]:
dados_cad_id_unicos[dados_cad_id_unicos.Anos_empregado < 0].shape

(75314, 18)

As colunas com o tipo **`float64`** parecem estar classificadas corretamente, mas a variável **`Anos_empregado`** está com um erro de preenchimento, onde temos 75.314 registros com o valor de -1000,7, chegando a distorcer a média geral do *dataset*.

Isso precisa ser tratado antes de prosseguirmos.

In [60]:
for var in cols_object:
    print(var,'- Valores:', str(np.sort(dados_cad_id_unicos[var].unique())))

Genero - Valores: ['H' 'M']
Categoria_de_renda - Valores: ['Associado comercial' 'Empregado' 'Estudante' 'Pensionista'
 'Servidor publico']
Grau_Escolaridade - Valores: ['Ensino fundamental' 'Ensino medio' 'Ensino superior'
 'Ensino superior incompleto' 'Pos graduacao']
Estado_Civil - Valores: ['Casado' 'Separado' 'Solteiro' 'Uniao estavel' 'Viuvo']
Moradia - Valores: ['Alugado' 'Apartamento comercial' 'Casa propria'
 'Cooperativa habitacional' 'Habitacao publica' 'Mora com os pais']
Ocupacao - Valores: ['Alta tecnologia' 'Baixa qualificacao' 'Construcao Civil' 'Contabilidade'
 'Corretor imobiliario' 'Cozinha' 'Equipe principal' 'Garcom' 'Gerencia'
 'Limpeza' 'Medicina' 'Motorista' 'Outros' 'RH' 'Secretariado' 'Seguranca'
 'Servico privado' 'TI' 'Venda']


Entre as colunas com o tipo **`object`** temos apenas um ajuste que poderia ser feito, já pensando na futura utilização de *Machine Learning*, que é transformar a variável **`Genero`** em numérica binária.

### **O que aprendemos nessa aula?**
---

>* **Mercado financeiro:**
>
>  * Pilares da cadeia de crédito;
>  * Fluxo de um pedido de crédito.
>
>* **Base de dados:**
>
>  * Exploração da base de clientes cadastrados e suas features;
>  * Verificação de dados duplicados e tratamento;
>  * Verificação de dados nulos.
---
